In [3]:
import ktrain
from ktrain import text as txt

In [4]:
TDATA = 'dataset/train.txt'
VDATA = 'dataset/test.txt'

In [5]:
# load data
(trn, val, preproc) = txt.entities_from_txt('dataset/train.txt', 'dataset/test.txt',                   
                                            use_char=True) # enable character embeddings

detected encoding: utf-8 (if wrong, set manually)
Number of sentences:  1
Number of words in the dataset:  11441
Tags: ['B-DLEG', 'I-LEI', 'I-LCOMP', 'I-EC', 'I-LDEL', 'B-LCOMP', 'I-DEC', 'I-DLEG', 'B-DEC', 'B-LDEL', 'B-DLEI', 'I-DLEI', 'B-None', 'B-EC', 'O', 'I-None', 'B-LEI']
Number of Labels:  17
Longest sentence: 314279 words


In [ ]:
# load model
model = txt.sequence_tagger('bilstm-crf', preproc)

In [7]:
# wrap model and data in ktrain.Learner object
learner = ktrain.get_learner(model, train_data=trn, val_data=val)

In [ ]:

# conventional training for 1 epoch using a learning rate of 0.001 (Keras default for Adam optmizer)
learner.fit(0.01, 2, cycle_len=5, checkpoint_folder='/tmp/saved_weights')

preparing training data ...done.
preparing validation data ...done.
Epoch 1/10


In [ ]:
learner.validate(class_names=preproc.get_classes())